In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from functools import reduce
import os

In [2]:
with open('/content/drive/MyDrive/The Problem: Classify Wikipedia Disease Articles/training/positive/AIDS') as fp:
  soup = BeautifulSoup(fp, 'html.parser')
  body = soup.select('h1#firstHeading')[0]
  print(body)

<h1 class="firstHeading" id="firstHeading" lang="en">HIV/AIDS</h1>


In [3]:
def get_heading_name(heading):
  return heading.select('h1#firstHeading')[0].get_text()


In [4]:
def get_doc(file_path):
  title=""
  content=""
  with open(file_path) as fp:
    soup = BeautifulSoup(fp, 'html.parser')
  return soup

In [5]:
#Headings
def TOC_Level1(cont):
  toc = []
  toc_content = cont.select('#toc li.toclevel-1 a')
  if len(toc_content) > 0:
    toc = [tocs['href'][1:len(tocs['href'])] for tocs in toc_content]
  return toc

In [6]:
# Section
def TOC_Level2(cont, toc):
  sec_info = cont.find_all(recursive=False)
  disease_info={}
  collecting = None
  for info in sec_info:
    toc_content=info.select('span.mw-headline')
    if len(toc_content) > 0:
      toc_text = toc_content[0].get_text()
      if len(toc) == 0 or toc_text in toc:
        collecting = toc_content[0].get_text()
        disease_info[collecting] = ""
      
    else:
      if collecting:
        toc_text_acc = disease_info[collecting]
        text = info.get_text()
        toc_text_acc = toc_text_acc +" "+ text
        disease_info[collecting] = toc_text_acc

  return disease_info

In [7]:
def Extract_all_info(file_path):
  info = {}
  soup = get_doc(file_path)
  disease_name = get_heading_name(soup)
  cont = soup.select('#mw-content-text')[0]
  toc = TOC_Level1(cont)
  disease_info = TOC_Level2(cont, toc)

  return {'disease_name':disease_name, 'toc':disease_info.keys(), 'disease_info': disease_info}

In [8]:
positive_data = os.fsencode('/content/drive/MyDrive/The Problem: Classify Wikipedia Disease Articles/training/positive')

In [12]:
def DataExtraction(x):
  disease_names = []
  tocs = []
  disease_info_list = []

  for file in os.listdir(x):
    file_path = os.path.join(positive_data, files[i])
    info = Extract_all_info(file_path)
    disease_names.append(info['disease_name'])
    tocs.append(info['toc'])
    disease_info_list.append(info['disease_info'])

  data_frame = {'disease_name':disease_names, 'extracted_headings':tocs, 'disease_info':disease_info_list}
  df = pd.DataFrame(data_frame)
  return df

In [13]:
df = DataExtraction(positive_data)
df.head()

,disease_name,extracted_headings,disease_info
0,Pipecolic acidemia,"(Characteristics, References)",{'Characteristics': ' The disorder is characte...
1,Pipecolic acidemia,"(Characteristics, References)",{'Characteristics': ' The disorder is characte...
2,Pipecolic acidemia,"(Characteristics, References)",{'Characteristics': ' The disorder is characte...
3,Pipecolic acidemia,"(Characteristics, References)",{'Characteristics': ' The disorder is characte...
4,Pipecolic acidemia,"(Characteristics, References)",{'Characteristics': ' The disorder is characte...


In [15]:
df.shape

(3692, 3)

In [16]:
df.to_csv('/content/drive/MyDrive/The Problem: Classify Wikipedia Disease Articles/training/ExtractedData.csv', header=True, index=False)